In [163]:
import pandas as pd
import warnings 
import re

In [164]:
def read_data():
    """
    Description:
    Read the data from the files and return the dataframes
    return: df, dfcc, dfc (dataframes) 
    """
    df = pd.read_csv('results/EntrevistaInicial.csv')
    dfcc = pd.read_csv('dataset/CentrosDeCostoEstado.csv')
    dfc = pd.read_csv('dataset/CentrosDeCosto.csv')
    return df, dfcc, dfc

In [165]:
def get_dict ():
    """
    Description: Return a dictionary with the states
    return: dict_Estados (dictionary)
    """

    dict_Mes ={1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'}
    return dict_Mes

In [166]:
def transform_data(df):
    """
    Description: Transform the data from the dataframe
    return: df (dataframe)
    """
    
    dict_Mes = get_dict()
    df['MesRegistro'] = df['MesRegistro'].map(dict_Mes)
    return df

In [167]:
def split_date(df):
    """
    Description: Split the date in year, month and day
    return: df (dataframe)
    """
    
    df['FechaRegistro'] = pd.to_datetime(df['FechaRegistro'])
    df['AñoRegistro'] = df['FechaRegistro'].dt.year
    df['MesRegistro'] = df['FechaRegistro'].dt.month
    df['DiaRegistro'] = df['FechaRegistro'].dt.day
    df = transform_data(df)
    return df

In [168]:
def sem(df , lis_1sem, lis_2sem):
    """
    Description: Create a column with the semester
    return: df (dataframe)
    """
    
    df['Semestre'] = 0
    for ind, val in df['MesRegistro'].items():
        if val in lis_1sem:
            df.loc[ind, 'Semestre'] = 1
        elif val in lis_2sem:
            df.loc[ind, 'Semestre'] = 2
    return df

In [169]:
def mod_cc(df, dictc, dictcc):
    """
    Description: Modify the column CentroCostoId
    return: df (dataframe)
    """
    
    df['CentroCostoId'] = df['CentroCostoId'].map(dictc)
    df['CentroCostoId'] = df['CentroCostoId'].map(dictcc)
    return df

In [170]:
def get_cc (df, dfcc, dfc):
    """
    Description: Get the dataframes and return the dataframe with the modified column CentroCostoId
    return: df (dataframe)
    """
    
    dictcc = dict(zip(dfcc['CENTRO'], dfcc['ADSCRIPCIÓN']))
    dictc = dict(zip(dfc['CentroCostoId'], dfc['CentroCostoClave']))
    df = mod_cc(df, dictc, dictcc)
    return df

In [171]:
def group_sust():
    """
    Description: Create a dictionary with the substances
    return: dict_sust_inverso (dictionary)
    """
    
    dict_sust = {'Tabaco':(1,2) , 'Alcohol':(3,4,5,84), 'Marihuana':(6,7,8,9,10,11,),'Inalables':(24,25,26,27,28) , 'Cocaina':(12,13,14,15,86) , 'Metanfetaminas': (16,17,18,19,85) , 'OtrosEstimulantes':(19,20,21,22,23), 'Extasis':(22,50,51,48), 'Benzodiacepinas': (52,53) , 'OtrosDepresores': (54,55,56), 'Alucinogenos': (29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47), 'Heroina': (60,61), 'OpiodesSinteticos': (62,63,64,65,66,67), 'OpioDerivados': (57,58,59), 'UtilidadMedica': (68,69,70,71,72,73,74,75), 'SustanciasDeAbuso': (76,77,78,79,80,81,82), 'Otros': (83,10000)}
    dict_sust_inverso = {}
    for sustancia, numeros in dict_sust.items():
        for numero in numeros:
            dict_sust_inverso[numero] = sustancia
    return dict_sust_inverso

In [172]:
def mod_sust (df):
    """
    Description: Modify the columns MayorImpactoSustanciaId
    return: df (dataframe)
    """
    
    dict_sust = group_sust()
    for col in df.columns:
        if col.startswith('MayorImpactoSustanciaId'):
            df[col] = df[col].map(dict_sust)
    for col in df.columns:
        if col.startswith('SustanciaI'):
            df[col] = df[col].map(dict_sust)
    return df

In [173]:
def mod_data(df, dfcc, dfc):
    """
    Description: Modify the data from the dataframe
    return: df (dataframe)
    """
    
    df = split_date(df)
    lista_1sem = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio']
    lista_2sem = ['Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
    df = sem(df, lista_1sem, lista_2sem)
    df = get_cc(df, dfcc, dfc)
    df = mod_sust(df)
    return df

In [174]:
warnings.filterwarnings("ignore")
df, dfcc , dfc = read_data()
df = mod_data(df, dfcc, dfc)

In [175]:
def get_total_cons (group, semestre, año):
    """
    Description: Get the total of consultations
    return: total (int)
    """
    
    list_total = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    group = group[(group['Semestre'] == semestre) & (group['AñoRegistro'] == año)]
    sustancia_a_indice = {
        'Tabaco': 0, 'Alcohol': 1, 'Marihuana': 2, 'Inalables': 3, 'Cocaina': 4,
        'Metanfetaminas': 5, 'OtrosEstimulantes': 6, 'Extasis': 7, 'Benzodiacepinas': 8,
        'OtrosDepresores': 9, 'Alucinogenos': 10, 'Heroina': 11, 'OpiodesSinteticos': 12,
        'OpioDerivados': 13, 'UtilidadMedica': 14, 'SustanciasDeAbuso': 15, 'Otros': 16
    }
    for ind, val in group['MayorImpactoSustanciaId'].items():
        if val in sustancia_a_indice:
            list_total[sustancia_a_indice[val]] += 1    
    return list_total

In [176]:
def get_total_cons_cc (group):
    """
    Description: Get the total of consultations by center
    return: list_total_cc (list)
    """
    
    list_total_cc = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    sustancia_a_indice = {
        'Tabaco': 0, 'Alcohol': 1, 'Marihuana': 2, 'Inalables': 3, 'Cocaina': 4,
        'Metanfetaminas': 5, 'OtrosEstimulantes': 6, 'Extasis': 7, 'Benzodiacepinas': 8,
        'OtrosDepresores': 9, 'Alucinogenos': 10, 'Heroina': 11, 'OpiodesSinteticos': 12,
        'OpioDerivados': 13, 'UtilidadMedica': 14, 'SustanciasDeAbuso': 15, 'Otros': 16
    }
    for ind, val in group['MayorImpactoSustanciaId'].items():
        if val in sustancia_a_indice:
            list_total_cc[sustancia_a_indice[val]] += 1
    return list_total_cc

In [177]:
def map_impacto(group , df):
    """
    Description: Map the impact of the drugs
    return: df_result (dataframe)
    """
    
    df_result = pd.DataFrame(columns = ['Semestre', 'Año', 'Unidad', 'Estado', 'Categoria', 'Tabaco', 'ArribaMediaTabaco', 'Alcohol', 'ArribaMediaAlcohol', 'Inalables', 'ArribaMediaInalables', 'Cocaina', 'ArribaMediaCocaina', 'Metanfetaminas', 'ArribaMediaMetanfetaminas', 'OtrosEstimulantes', 'ArribaMediaOtrosEstimulantes', 'Extasis', 'ArribaMediaExtasis', 'Benzodiacepinas', 'ArribaMediaBenzodiacepinas', 'OtrosDepresores', 'ArribaMediaOtrosDepresores', 'Alucinogenos', 'ArribaMediaAlucinogenos', 'Heroina', 'ArribaMediaHeroina', 'OpiodesSinteticos', 'ArribaMediaOpiodesSinteticos', 'OpioDerivados', 'ArribaMediaOpioDerivados', 'UtilidadMedica', 'ArribaMediaUtilidadMedica', 'SustanciasDeAbuso', 'ArribaMediaSustanciasDeAbuso', 'Otros', 'ArribaMediaOtros'])
    semestre = group['Semestre'].iloc[0]
    año = group['AñoRegistro'].iloc[0]
    list_total = get_total_cons(df, semestre, año)
    list_totalcc = get_total_cons_cc(group)
    df_result['Semestre'][0] = group['Semestre'].iloc[0]
    df_result['Año'][0] = group['AñoRegistro'].iloc[0]
    df_result['Unidad'][0] = 'Total'
    df_result['Estado'][0] = group['CentroCostoId'].iloc[0]
    df_result['Categoria'][0] = 'DrogaImpacto'
    sustancias = ['Tabaco', 'Alcohol', 'Inalables', 'Cocaina', 'Metanfetaminas', 'OtrosEstimulantes', 'Extasis', 'Benzodiacepinas', 'OtrosDepresores', 'Alucinogenos', 'Heroina', 'OpiodesSinteticos', 'OpioDerivados', 'UtilidadMedica', 'SustanciasDeAbuso', 'Otros']
    for i, sustancia in enumerate(sustancias):
        df_result[sustancia][0] = list_totalcc[i]
        df_result[f'ArribaMedia{sustancia}'][0] = 'Si' if int(list_totalcc[i]) > int(list_total) / 32 else 'No'
    return df_result


In [178]:
def map_impacto(group , df):
    """
    Description: Map the impact of the drugs
    return: df_result (dataframe)
    """
    
    df_result = pd.DataFrame(columns = ['Semestre', 'Año', 'Unidad', 'Estado', 'Categoria', 'Tabaco', 'ArribaMediaTabaco', 'Alcohol', 'ArribaMediaAlcohol', 'Inalables', 'ArribaMediaInalables', 'Cocaina', 'ArribaMediaCocaina', 'Metanfetaminas', 'ArribaMediaMetanfetaminas', 'OtrosEstimulantes', 'ArribaMediaOtrosEstimulantes', 'Extasis', 'ArribaMediaExtasis', 'Benzodiacepinas', 'ArribaMediaBenzodiacepinas', 'OtrosDepresores', 'ArribaMediaOtrosDepresores', 'Alucinogenos', 'ArribaMediaAlucinogenos', 'Heroina', 'ArribaMediaHeroina', 'OpiodesSinteticos', 'ArribaMediaOpiodesSinteticos', 'OpioDerivados', 'ArribaMediaOpioDerivados', 'UtilidadMedica', 'ArribaMediaUtilidadMedica', 'SustanciasDeAbuso', 'ArribaMediaSustanciasDeAbuso', 'Otros', 'ArribaMediaOtros'])
    if df_result.empty:
        df_result.loc[0] = 0
    semestre = group['Semestre'].iloc[0]
    año = group['AñoRegistro'].iloc[0]
    list_total = get_total_cons(df, semestre, año)
    list_totalcc = get_total_cons_cc(group)
    df_result['Semestre'][0] = group['Semestre'].iloc[0]
    df_result['Año'][0] = group['AñoRegistro'].iloc[0]
    df_result['Unidad'][0] = 'Total'
    df_result['Estado'][0] = group['CentroCostoId'].iloc[0]
    df_result['Categoria'][0] = 'DrogaImpacto'

    sustancias = ['Tabaco', 'Alcohol', 'Inalables', 'Cocaina', 'Metanfetaminas', 'OtrosEstimulantes', 'Extasis', 'Benzodiacepinas', 'OtrosDepresores', 'Alucinogenos', 'Heroina', 'OpiodesSinteticos', 'OpioDerivados', 'UtilidadMedica', 'SustanciasDeAbuso', 'Otros']
    df_result.loc[0, sustancias] = list_totalcc[:len(sustancias)]
    
    sustancias = ['Tabaco', 'Alcohol', 'Inalables', 'Cocaina', 'Metanfetaminas', 'OtrosEstimulantes', 'Extasis', 'Benzodiacepinas', 'OtrosDepresores', 'Alucinogenos', 'Heroina', 'OpiodesSinteticos', 'OpioDerivados', 'UtilidadMedica', 'SustanciasDeAbuso', 'Otros']
    for i, sustancia in enumerate(sustancias):
        if df_result[sustancia][0] > list_total[i] / 32:
            df_result[f'ArribaMedia{sustancia}'][0] = 1
    return df_result

In [179]:
warnings.filterwarnings("ignore")
df_result = pd.DataFrame()
for val , group in df.groupby(['Semestre', 'AñoRegistro', 'CentroCostoId']):
    df_result = pd.concat([df_result, map_impacto(group, df)])
  


In [180]:
def elim_col (df):
    df['SustanciaId0'] = df['SustanciaId']
    df['ComunAbstinenciaId0'] = df['ComunAbstinenciaId']
    df.drop(columns = ['SustanciaId', 'ComunAbstinenciaId'], inplace = True)
    return df

In [181]:
def get_total_cons_UM (group, semestre, año):
    """
    Description: Get the total of consultations by unit
    return: list_total (list)
    """
    
    list_total = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    group = group[(group['Semestre'] == semestre) & (group['AñoRegistro'] == año)]
    sustancia_a_indice = {
        'Tabaco': 0, 'Alcohol': 1, 'Marihuana': 2, 'Inalables': 3, 'Cocaina': 4,
        'Metanfetaminas': 5, 'OtrosEstimulantes': 6, 'Extasis': 7, 'Benzodiacepinas': 8,
        'OtrosDepresores': 9, 'Alucinogenos': 10, 'Heroina': 11, 'OpiodesSinteticos': 12,
        'OpioDerivados': 13, 'UtilidadMedica': 14, 'SustanciasDeAbuso': 15, 'Otros': 16
    }
    for col in group.columns:
        if col.startswith('SustanciaI'):
            exp = re.search(r'\d+', col)
            if exp:
                for ind , val in group[col].items():
                    if val in sustancia_a_indice:
                        if group['ComunAbstinenciaId'+exp.group(0)][ind] in range(1, 4):
                            list_total[sustancia_a_indice[val]] += 1
    return list_total

In [182]:
def get_total_cons_cc_UM (group):
    """
    Description: Get the total of consultations by center
    return: list_total_cc (list)
    """
    
    list_total_cc = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    sustancia_a_indice = {
        'Tabaco': 0, 'Alcohol': 1, 'Marihuana': 2, 'Inalables': 3, 'Cocaina': 4,
        'Metanfetaminas': 5, 'OtrosEstimulantes': 6, 'Extasis': 7, 'Benzodiacepinas': 8,
        'OtrosDepresores': 9, 'Alucinogenos': 10, 'Heroina': 11, 'OpiodesSinteticos': 12,
        'OpioDerivados': 13, 'UtilidadMedica': 14, 'SustanciasDeAbuso': 15, 'Otros': 16
    }
    for col in group.columns:
        if col.startswith('SustanciaI'):
            exp = re.search(r'\d+', col)
            if exp:
                for ind, val in group[col].items():
                    if val in sustancia_a_indice:
                        if group['ComunAbstinenciaId'+exp.group(0)][ind] in range(1, 4):
                            list_total_cc[sustancia_a_indice[val]] += 1
    return list_total_cc

In [183]:
def map_impacto(group , df):
    """
    Description: Map the impact of the drugs
    return: df_result (dataframe)
    """
    
    df_result = pd.DataFrame(columns = ['Semestre', 'Año', 'Unidad', 'Estado', 'Categoria', 'Tabaco', 'ArribaMediaTabaco', 'Alcohol', 'ArribaMediaAlcohol', 'Inalables', 'ArribaMediaInalables', 'Cocaina', 'ArribaMediaCocaina', 'Metanfetaminas', 'ArribaMediaMetanfetaminas', 'OtrosEstimulantes', 'ArribaMediaOtrosEstimulantes', 'Extasis', 'ArribaMediaExtasis', 'Benzodiacepinas', 'ArribaMediaBenzodiacepinas', 'OtrosDepresores', 'ArribaMediaOtrosDepresores', 'Alucinogenos', 'ArribaMediaAlucinogenos', 'Heroina', 'ArribaMediaHeroina', 'OpiodesSinteticos', 'ArribaMediaOpiodesSinteticos', 'OpioDerivados', 'ArribaMediaOpioDerivados', 'UtilidadMedica', 'ArribaMediaUtilidadMedica', 'SustanciasDeAbuso', 'ArribaMediaSustanciasDeAbuso', 'Otros', 'ArribaMediaOtros'])
    if df_result.empty:
        df_result.loc[0] = 0
    semestre = group['Semestre'].iloc[0]
    año = group['AñoRegistro'].iloc[0]
    list_total = get_total_cons_UM(df, semestre, año)
    list_totalcc = get_total_cons_cc_UM(group)
    df_result['Semestre'][0] = group['Semestre'].iloc[0]
    df_result['Año'][0] = group['AñoRegistro'].iloc[0]
    df_result['Unidad'][0] = 'Total'
    df_result['Estado'][0] = group['CentroCostoId'].iloc[0]
    df_result['Categoria'][0] = 'DrogaImpacto'

    sustancias = ['Tabaco', 'Alcohol', 'Inalables', 'Cocaina', 'Metanfetaminas', 'OtrosEstimulantes', 'Extasis', 'Benzodiacepinas', 'OtrosDepresores', 'Alucinogenos', 'Heroina', 'OpiodesSinteticos', 'OpioDerivados', 'UtilidadMedica', 'SustanciasDeAbuso', 'Otros']
    df_result.loc[0, sustancias] = list_totalcc[:len(sustancias)]
    
    sustancias = ['Tabaco', 'Alcohol', 'Inalables', 'Cocaina', 'Metanfetaminas', 'OtrosEstimulantes', 'Extasis', 'Benzodiacepinas', 'OtrosDepresores', 'Alucinogenos', 'Heroina', 'OpiodesSinteticos', 'OpioDerivados', 'UtilidadMedica', 'SustanciasDeAbuso', 'Otros']
    for i, sustancia in enumerate(sustancias):
        if df_result[sustancia][0] > list_total[i] / 32:
            df_result[f'ArribaMedia{sustancia}'][0] = 1
    return df_result

In [184]:
df = elim_col(df)
warnings.filterwarnings("ignore")
df_result_UM = pd.DataFrame()
for val , group in df.groupby(['Semestre', 'AñoRegistro', 'CentroCostoId']):
    df_result_UM = pd.concat([df_result_UM, map_impacto(group, df)])
  

In [185]:
df_final = pd.concat([df_result, df_result_UM])
df_final.to_csv('results/ImpactoDrogas.csv', index = False, encoding='utf-8')